In [ ]:
from pprint import pprint
import json
import pandas as pd
from TraceLens import TreePerfAnalyzer

In [ ]:
# replace by your profile path, it can be a single rank profile from a multi gpu run as well
path = '/path/to/profile.json'
perf_analyzer = TreePerfAnalyzer.from_file(path)

In [ ]:
aten_mm_events = [event for event in perf_analyzer.tree.events if event['name'] =='aten::mm']
aten_mm_event = aten_mm_events[123]
pprint(aten_mm_event)

In [ ]:
import torch

dict_profile2torchdtype = {
    'c10::BFloat16': torch.bfloat16,
    'c10::Half': torch.half
}

def get_tensors_from_args(args, device):
    tensors = []
    for i in range(len(args['Input Dims'])):
        shape = tuple(args['Input Dims'][i])
        dtype = dict_profile2torchdtype[args['Input type'][i]]
        stride = tuple(args['Input Strides'][i])
        tensor = torch.randn(shape, device=device, dtype=dtype)
        tensor = tensor.as_strided(size=shape, stride=stride)
        tensors.append(tensor)
    return tensors

def replay_aten_mm(event, device):
    tensors = get_tensors_from_args(event['args'], device)
    result = torch.ops.aten.mm(tensors[0], tensors[1])

In [ ]:
# replay the aten::mm event
device = 'cuda:0'
replay_aten_mm(aten_mm_event, device)
